In [1]:
%cd ..

/Users/lyhai/Documents/GitHub/RAG systems for research 


/Users/lyhai/.pyenv/versions/3.10.4/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from src.ingest import *
raw_texts = load_documents()
raw_texts

/Users/lyhai/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ignoring wrong pointing object 1549 0 (offset 0)
Ignoring wrong pointing object 1668 0 (offset 0)
Ignoring wrong pointing object 1680 0 (offset 0)
Ignoring wrong pointing object 1682 0 (offset 0)
Ignoring wrong pointing object 1683 0 (offset 0)
Ignoring wrong pointing object 1684 0 (offset 0)
Ignoring wrong pointing object 1685 0 (offset 0)
Ignoring wrong pointing object 1686 0 (offset 0)
Ignoring wrong pointing object 1691 0 (offset 0)
Ignoring wrong pointing object 1692 0 (offset 0)
Ignoring wrong pointing object 1693 0 (offset 0)
Ignoring wrong pointing object 1694 0 (offset 0)
Ignoring wrong pointing object 1695 0 (offset 0)
Ignoring wrong pointing object 1703 0 (offset 0)
Ignoring wrong pointing object 1708 0 (offse

[Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'Elsevier', 'creationdate': '2024-10-22T16:43:59+00:00', 'crossmarkdomains[1]': 'elsevier.com', 'crossmarkmajorversiondate': '2010-04-23', 'creationdate--text': '22nd October 2024', 'elsevierwebpdfspecifications': '7.0', 'robots': 'noindex', 'moddate': '2025-02-14T12:52:31-05:00', 'author': 'Min Joo Kim', 'doi': '10.1016/j.ijhcs.2024.103385', 'title': 'Comparing typing methods for uppercase input in virtual reality: Modifier Key vs. alternative approaches', 'keywords': 'Virtual reality (VR),Virtual environment (VE),Typing interaction,User experience (UX),Modifier key', 'subject': 'International Journal of Human - Computer Studies, 193 (2025) 103385. doi:10.1016/j.ijhcs.2024.103385', 'crossmarkdomains[2]': 'sciencedirect.com', 'crossmarkdomainexclusive': 'true', 'source': './data/Comparing typing methods for uppercase input in virtual reality- Modifier Key vs. alternative approaches.pdf', 'total_pages': 10, 

In [3]:
texts = chunk_text(raw_texts)
len(texts)

134

In [5]:
texts

[Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'Elsevier', 'creationdate': '2024-10-22T16:43:59+00:00', 'crossmarkdomains[1]': 'elsevier.com', 'crossmarkmajorversiondate': '2010-04-23', 'creationdate--text': '22nd October 2024', 'elsevierwebpdfspecifications': '7.0', 'robots': 'noindex', 'moddate': '2025-02-14T12:52:31-05:00', 'author': 'Min Joo Kim', 'doi': '10.1016/j.ijhcs.2024.103385', 'title': 'Comparing typing methods for uppercase input in virtual reality: Modifier Key vs. alternative approaches', 'keywords': 'Virtual reality (VR),Virtual environment (VE),Typing interaction,User experience (UX),Modifier key', 'subject': 'International Journal of Human - Computer Studies, 193 (2025) 103385. doi:10.1016/j.ijhcs.2024.103385', 'crossmarkdomains[2]': 'sciencedirect.com', 'crossmarkdomainexclusive': 'true', 'source': './data/Comparing typing methods for uppercase input in virtual reality- Modifier Key vs. alternative approaches.pdf', 'total_pages': 10, 

In [6]:
embedder = get_embedders()

embeddings = embedder.encode([t.page_content for t in texts], convert_to_numpy=True)
embeddings.shape

(134, 384)